# Проект [SF-DST Car Price prediction] Прогнозирование стоимости автомобиля по характеристикам

## Импорт библиотек

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
# InteractiveShell.ast_node_interactivity = "last_expr"

In [ ]:
import requests
# from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_selection import f_classif, mutual_info_classif
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm
from catboost import CatBoostRegressor

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.tree import ExtraTreeRegressor

from sklearn import metrics  # инструменты для оценки точности модели

In [ ]:
RANDOM_SEED = 42
random_state = RANDOM_SEED

# Setup

In [ ]:
DIR_TRAIN  = '../input/train-data-autoru/' # подключил к ноутбуку свой внешний датасет
DIR_TEST   = '../input/sf-dst-car-price/'
VAL_SIZE   = 0.33   # 33%
N_FOLDS    = 5

# CATBOOST
ITERATIONS = 2000
LR         = 0.1

In [ ]:
def get_boxplot(df, column):
    fig, ax = plt.subplots(figsize = (14, 4))
    sns.boxplot(x=column, y='price', 
                data=df.loc[df.loc[:, column].isin(df.loc[:, column].value_counts().index[:10])],
               ax=ax)
    plt.xticks(rotation=45)
    ax.set_title('Boxplot for ' + column)
    plt.show()

In [ ]:
def heat_map(df_bool):
    fig, ax = plt.subplots(figsize=(20, 12))
    sns_heatmap = sns.heatmap(df_bool, yticklabels=False, cbar=False, cmap='viridis')

## Чтение данных

In [ ]:
df_train = pd.read_csv(DIR_TRAIN + 'train.csv.gzip', compression='gzip')
df_test = pd.read_csv(DIR_TEST + 'test.csv')
sample_submission = pd.read_csv(DIR_TEST + 'sample_submission.csv')

## Данные для обучения

Данные получены с сайта auto.ru с помощью https://github.com/kopagm/skillfactory_rds/tree/master/module_6/utils/autoru

### Предобработка данных для обучения

Отбрасываем дубликаты и строки с пропущенной ценой.

In [ ]:
df_train = df_train.drop_duplicates()
df_train.info()

In [ ]:
heat_map(df_train.isna())

In [ ]:
df_train = df_train.dropna(subset=['price'])
df_train.info()

Пропуски остались только в "Владение"

## Объединение тренировочной выборки и выборки для предсказания

In [ ]:
df_train = df_train.convert_dtypes()
df_train.info()

In [ ]:
df_test = df_test.convert_dtypes()
df_test.info()

In [ ]:
# ВАЖНО! дряastypeректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1  # помечаем где у нас трейн
df_test['sample'] = 0  # помечаем где у нас тест
df_test['price'] = 0 #

df = df_test.append(df_train, sort=False).reset_index(drop=True)  # объединяем

In [ ]:
df.info()

## Предобработка

### Пропуски

In [ ]:
heat_map(df.isna())

Пропуски только в "Владение" и в "id" для тренировочной выборки. "id" не несет информации и не понадобится.

In [ ]:
df.nunique()

### Преобразования значений и переименование полей

In [ ]:
def preproc_data(df_input):
    def string_to_float(a):
        try:
            return float(a)
        except ValueError:
            return None

    def get_list_from_equip(eqips):
        result = []
        for eq in eqips:
            result += eq['values']
        return result

    df = df_input.copy()
    df['enginePower'] = df['enginePower'].apply(lambda x: str.split(x)[0])
    
    df.loc[df['sample'] == 0,
           'equip'] = df.loc[df['sample'] == 0,
                             'Комплектация'].apply(lambda x: json.loads(str.replace(x, "\'", '')))
    df.loc[df['sample'] == 1,
           'equip'] = df.loc[df['sample'] == 1,
                             'Комплектация'].apply(lambda x: json.loads(str.replace(x, "'", '"')))
    # преведение к единому виду с train
    df.loc[df['sample'] == 0, 'equip'] = df.loc[df['sample'] == 0,
                                                'equip'].apply(lambda x: x[0] if len(x) else [])
    df['equip'] = df['equip'].apply(get_list_from_equip)
    
    df['engineDisplacement'] = df['engineDisplacement'].apply(lambda x: str.split(x)[0])
    df['engineDispl'] = df['engineDisplacement'].apply(string_to_float)
    df.drop(columns=['engineDisplacement'])
    df['Привод'] = df['Привод'].str.lower()
    df['Владельцы'] = df['Владельцы'].apply(lambda x: str.split(x)[0])

    df['price'] = df['price'].apply(lambda x: np.int(x))

    # drop columns
    df.drop(columns=['engineDisplacement', 'Комплектация', 'id', 'Состояние'], inplace=True)
    # fill na
    df['engineDispl'].fillna(df['engineDispl'].mean())
    return df

In [ ]:
df = preproc_data(df)

In [ ]:
columns = df.columns
df.loc[0:5,columns[:]]

In [ ]:
df.info()

## Классификация параметров

In [ ]:
columns = df.columns

In [ ]:
bin_cols = ['Руль', 'ПТС']
cat_cols = ['bodyType', 'brand', 'color', 'fuelType', 'modelDate', 'name',
       'numberOfDoors', 'productionDate', 'vehicleConfiguration',
       'vehicleTransmission', 'enginePower',
       'Привод', 'Владельцы', 'Таможня', 'Владение', 'engineDispl']
num_cols = ['mileage', 'price']


## Визуализация параметров

### Категориальные

In [ ]:
for col in cat_cols+bin_cols:
#     get_boxplot(df[df['sample'] == 1], col)
    get_boxplot(df, col)

Часто встречающиеся значения "name" присутсвуют в тесте, но отсутствуют в трейне (отсутствует цена на диаграмме).

### Числовые

In [ ]:
sns.jointplot(x='mileage', y='price', data=df_train)

In [ ]:
df[num_cols].describe()

In [ ]:
def get_boxplot_single(df, column):
    fig, ax = plt.subplots()
    sns.boxplot(y=column, 
                data=df,
               ax=ax)
    ax.set_title('Boxplot for ' + column)
    plt.show()

In [ ]:
def get_hist(df, column):
    fig, ax = plt.subplots()
    df[column].plot.hist()
    ax.set_title('Histogram for ' + column)
    plt.show()

In [ ]:
for col in num_cols:
    if col == 'price':
        df_g = df.loc[df['sample'] == 1]
    else:
        df_g = df
    get_hist(df_g, col)
    get_boxplot_single(df_g, col)

Распределения имеют выбросы и длинные хвосты с права.

In [ ]:
df.loc[df['mileage'] > 5e5, ['mileage', 'price', 'sample']]

Можно попробовать и с удалением выбросов 'mileage' и с ними. Пока оставим все значения.

## Создание новых параметров

### Новые параметры на основании комплектаций (equip)

Создадим новые параметры из спсиков параметра 'equip'

In [ ]:
def get_equip_features(df):
    def get_equips(df):
        res = set()
        for eq in df['equip']:
            res.update(set(eq))
        return res
    
    def get_equips(df):
        res = set()
        for eq in df['equip']:
            res.update(set(eq))
        return res
    
    for feature in get_equips(df):
        df['eq_'+feature] = df['equip'].apply(lambda x: 1 if feature in x else 0)

In [ ]:
get_equip_features(df)

In [ ]:
df.columns

### brand 

Пока в выборке только один бренд - удаляем.

### vehicleConfiguration 

vehicleConfiguration 
повторяет информцию из других параметров. Удалим
<!-- Выберем 10 наиболее частых и остальные объединим. -->

### name 

In [ ]:
a = df.name.value_counts().values
pd.DataFrame(a, columns=['name']).plot(xlabel='N name', ylabel='counts')

In [ ]:
def label_top_values(df, col, n):
    top_values = list(df[col].value_counts().head(n).index)
    return df[col].apply(lambda x: x if x in top_values else 'other')

In [ ]:
label_top_values(df, 'name', 200).value_counts()

Велика чаcть состоящая из редких значений. Пока не будем использовать параметр.

### Удаление неиспользуемых параметров

In [ ]:
df.drop(columns=['equip', 'Владение', 'Таможня', 'brand', 'description',
                 'name', 'vehicleConfiguration'], inplace=True)
cat_cols = list(set(cat_cols).difference(set(['Владение', 'Таможня', 'brand',
                                              'name', 'vehicleConfiguration'])))
num_cols.remove('price')

In [ ]:
cat_cols_copy = cat_cols
cat_cols

### Кодирование бинарных

In [ ]:
label_encoder = LabelEncoder()
for column in bin_cols:
    df[column] = label_encoder.fit_transform(df[column])

### Dummy-кодирование

In [ ]:
df = pd.get_dummies(df, columns=cat_cols, dummy_na=False)

### Стандартизация числовых переменных

In [ ]:
df[num_cols] = pd.DataFrame(StandardScaler().fit_transform(df[num_cols]),
                            columns=num_cols)

## ML

### Подготовка данных

In [ ]:
X = df[df['sample'] == 1].drop(columns=['price', 'sample'])
X
y = df[df['sample'] == 1].price
y

## Train Split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=VAL_SIZE, shuffle=True, random_state=RANDOM_SEED)

## Модели

### CatBoost

In [ ]:
model = CatBoostRegressor(iterations = ITERATIONS,
                          learning_rate = LR,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE']
                         )
model.fit(X_train, y_train,
#          cat_features=cat_features_ids,
         eval_set=(X_test, y_test),
         verbose_eval=100,
         use_best_model=True,
         plot=True
         )

In [ ]:
def pr_metrics(y_true, y_pred):
    def mean_absolute_percentage_error(y_true, y_pred): 
        y_true, y_pred = np.array(y_true), np.array(y_pred)
        return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

    mae = metrics.mean_absolute_error(y_true, y_pred)
    mape = mean_absolute_percentage_error(y_true, y_pred)
    print('mae =',round(mae), ', mape =', mape)

In [ ]:
y_pred = model.predict(X_test)
pr_metrics(y_test, y_pred)

### LinearRegression

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
pr_metrics(y_test, y_pred)

### RandomForestRegressor

In [ ]:
model = RandomForestRegressor(
    n_estimators=4000,
    criterion='mse',
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    min_weight_fraction_leaf=0.0,
    max_features='auto',
    max_leaf_nodes=None,
    min_impurity_decrease=0.0,
    min_impurity_split=None,
    bootstrap=True,
    oob_score=False,
    n_jobs=-1,
    random_state=42,
    verbose=0,
    warm_start=False,
    ccp_alpha=0.0,
    max_samples=None,)
model = RandomForestRegressor()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
pr_metrics(y_test, y_pred)

#### Значимость параметров

In [ ]:
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(model.feature_importances_, index=X_train.columns)
feat_importances.nlargest(15).plot(kind='barh')

### Стакинг

In [ ]:
# https://github.com/Dyakonov/ml_hacks/blob/master/dj_stacking.ipynb

from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict

class DjStacking(BaseEstimator, ClassifierMixin):  
    """Стэкинг моделей scikit-learn"""

    def __init__(self, models, ens_model):
        """
        Инициализация
        models - базовые модели для стекинга
        ens_model - мета-модель
        """
        self.models = models
        self.ens_model = ens_model
        self.n = len(models)
        self.valid = None
        
    def fit(self, X, y=None, p=0.25, cv=3, err=0.001, random_state=None):
        """
        Обучение стекинга
        p - в каком отношении делить на обучение / тест
            если p = 0 - используем всё обучение!
        cv  (при p=0) - сколько фолдов использовать
        err (при p=0) - величина случайной добавки к метапризнакам
        random_state - инициализация генератора
            
        """
        if (p > 0): # делим на обучение и тест
            # разбиение на обучение моделей и метамодели
            train, valid, y_train, y_valid = train_test_split(X, y, test_size=p, random_state=random_state)
            
            # заполнение матрицы для обучения метамодели
            self.valid = np.zeros((valid.shape[0], self.n))
            for t, clf in enumerate(self.models):
                clf.fit(train, y_train)
                self.valid[:, t] = clf.predict(valid)
                
            # обучение метамодели
            self.ens_model.fit(self.valid, y_valid)
            
        else: # используем всё обучение
            
            # для регуляризации - берём случайные добавки
            self.valid = err*np.random.randn(X.shape[0], self.n)
            
            for t, clf in enumerate(self.models):
                # это oob-ответы алгоритмов
                self.valid[:, t] = self.valid[:, t] + cross_val_predict(clf, X, y, cv=cv, n_jobs=-1, method='predict')
                # но сам алгоритм надо настроить
                clf.fit(X, y)
            
            # обучение метамодели
            self.ens_model.fit(self.valid, y)  
            

        return self
    


    def predict(self, X, y=None):
        """
        Работа стэкинга
        """
        # заполение матрицы для мета-классификатора
        X_meta = np.zeros((X.shape[0], self.n))
        
        for t, clf in enumerate(self.models):
            X_meta[:, t] = clf.predict(X)
        
        a = self.ens_model.predict(X_meta)
        
        return (a)

In [ ]:
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.linear_model import LogisticRegression

knn1 = KNeighborsRegressor(n_neighbors=3)
knn2 = KNeighborsRegressor(n_neighbors=10)
rg0 = Ridge(alpha=0.01, random_state=random_state)
rg1 = Ridge(alpha=1.1, random_state=random_state)
rg2 = Ridge(alpha=100.1, random_state=random_state)
rf1 = RandomForestRegressor(n_estimators=100, max_depth=1, random_state=random_state)
rf2 = RandomForestRegressor(n_estimators=100, max_depth=5, random_state=random_state)
ext = ExtraTreesRegressor(n_estimators=300, n_jobs=-1, random_state=random_state)
lgr1 = LogisticRegression(C=0.001, penalty='l1', solver='saga', multi_class='ovr', max_iter=1000, random_state=42)
lgr2 = LogisticRegression(C=0.001, penalty='l2', random_state=42)
lgr3 = LogisticRegression(C=0.001, penalty='l2', solver='saga', multi_class='ovr', max_iter=1000, random_state=42)
cbr = CatBoostRegressor(iterations = ITERATIONS,
                          learning_rate = LR,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE']
                         )

In [ ]:
models = [knn1, knn2, rg1, rg2, rf1, rf2, ext, lgr1, lgr2, cbr]
ens_model = Ridge()

In [ ]:
s2 = DjStacking(models, ens_model)
s2.fit(X_train, y_train, p=-1, cv=7)

In [ ]:
y_pred = s2.predict(X_test)
pr_metrics(y_test, y_pred)

In [ ]:
s2 = DjStacking(models, ens_model)
s2.fit(X_train, y_train, p=-1, cv=5)

In [ ]:
y_pred = s2.predict(X_test)
pr_metrics(y_test, y_pred)

In [ ]:
best_model = s2

## Submission

In [ ]:
test_data = df[df['sample'] == 0].drop(columns=['price', 'sample'])

In [ ]:
test_data.sample(10)

In [ ]:
test_data.shape, df_test.shape, X_test.shape

In [ ]:
sample_submission
sample_submission.dtypes

In [ ]:
predict_submission = best_model.predict(test_data)

In [ ]:
submit = df_test.loc[:, ['id']]
predict_submission.shape
submit

In [ ]:
submit['price'] = predict_submission

In [ ]:
submit
submit.dtypes

In [ ]:
submit.sample(10)

In [ ]:
submit.to_csv('submission.csv', index=False)

## Заключение

### Итог

Лучший результат дал стакинг с числом фолдов 5

### Что можно улучшить

- Попробовать другой набор моделей для стакинга.
- Векторизировать и использовать текст описания
- Добавить пераметры на основании "name"
- Попробовать добавление в трейн данных с другим брендом